# Generate Questions and Errors from Biblical Data
I used GPT to create a prompt for me.  See this here https://chat.openai.com/share/37fce0d6-a384-4181-a2bf-6b2a18fd96bb



First, set your `OPENAI_API_KEY` environment variable. This allows you to securely authenticate with the OpenAI API, without worrying about leaving the value in your code. You can find your API key in your [OpenAI dashboard](https://help.openai.com/en/articles/4936850-where-do-i-find-my-secret-api-key).

In [2]:
import getpass, os, openai

secret_key = getpass.getpass('Enter OpenAI secret key: ') 
os.environ['OPENAI_API_KEY'] = secret_key

MODEL = 'gpt-3.5-turbo'





In [3]:
# If using the azure API endpoint, use the Azure instead
if not secret_key.startswith('sk-'):
    print("Using Azure API endpoint")
    openai.api_type = "azure"
    openai.api_base = os.environ['OPENAI_API_BASE'] = 'https://americasopenai.azure-api.net/'
    openai.api_version = os.environ['OPENAI_API_VERSION'] = '2023-05-15'
    openai.api_key = secret_key
    
    MODEL = 'gpt-35-turbo-16k'


Using Azure API endpoint


## Generate questions and answers from biblical data
Have GPT take on a role of linguistical biblical expert


In [14]:
role = """
As an AI trained in Biblical Greek and Hebrew, your role is to assist with quality control in Bible translation by generating a set of questions aimed at identifying potential errors in new translations. 

You'll be given Bible verses in their original languages and your task is to produce a structured JSON array that includes the verse reference, quality control questions, the correct answer to those questions, 
and examples of bad translations with the wrong answers they would yield.

 - Each example should be unique
 - Each example should be a plausible error that a human translator might make
 - The error must be very obvious and true across multiple languages.  Semantically similar words are not errors as many languages don't have the exact same word.  For instance "the message" and "the Word" may be the same.
 - The question should be simple enough an AI NLP model can answer it

"""
bad_examples = """
[{'verse': 'John 1:1',
  'question': 'Who was in the beginning?',
  'answer': 'The Word',
  'examples': {'bad_translation': 'In the start, the message was present.',
   'wrong_answer': 'the message'}},
 {'verse': 'John 1:1',
  'question': 'What was with God in the beginning?',
  'answer': 'The Word',
  'examples': {'bad_translation': 'In the outset, the communication was alongside God.',
   'wrong_answer': 'the communication'}},
 {'verse': 'John 1:1',
  'question': 'What was God?',
  'answer': 'The Word',
  'examples': {'bad_translation': 'In the beginning, God was a word.',
   'wrong_answer': 'a word'}}]
"""

fixed_examples = """
[{'verse': 'John 1:1',
  'question': 'Who was in the beginning?',
  'answer': 'The Word',
  'examples': {'bad_translation': 'In the start, the book was present.',
   'wrong_answer': 'the book'}},
 {'verse': 'John 1:1',
  'question': 'What was with God in the beginning?',
  'answer': 'The Word',
  'examples': {'bad_translation': 'In the outset, the speaker was alongside God.',
   'wrong_answer': 'the speaker'}},
 {'verse': 'John 1:1',
  'question': 'What was God?',
  'answer': 'The Word',
  'examples': {'bad_translation': 'In the beginning, God was a letter.',
   'wrong_answer': 'a letter'}}]
"""

examples = """
[
  {
    "verse": "Genesis 1:1",
    "question": "Who created the heavens and the earth?",
    "answer": "God",
    "examples": {
      "bad_translation": "At the start, gods made sky and rocks.",
      "wrong_answer": "gods"
    }
  },
  {
    "verse": "Genesis 1:1",
    "question": "What did God create in the beginning?",
    "answer": "The heavens and the earth",
    "examples": {
      "bad_translation": "In the outset, divinities fashioned the atmosphere and stones.",
      "wrong_answer": "atmosphere and stones"
    }
  },
  {
    "verse": "Genesis 1:1",
    "question": "When did God create?",
    "answer": "In the beginning",
    "examples": {
      "bad_translation": "Once upon a time, a deity brought forth the skies and the land.",
      "wrong_answer": "Once upon a time"
    }
  }
]
"""

In [18]:
import json
def query(verse="Jn 3:16", total=3):
    messages = [
        {"role":"system","content": role},
        {"role":"user","content": "Any"},
        {"role":"system","content": bad_examples},
        {"role":"user","content": "That is wrong!  Fix it!  The message and the communication carry close enough meaning to the Word that they are not errors.  word in lowercase is not an error b/c some languages don't have case like that.  I want errors that are obvious and true across multiple languages."},
        {"role":"system","content": fixed_examples},
        {"role":"user","content": "Genesis 1:1"},
        {"role":"system","content": examples},
    ]
    if not total == 3:
        messages.append({"role":"user","content": f"Use that format but I want {total} examples for {verse}"})
    else:
        messages.append({"role":"user","content": verse})
        
    response = openai.ChatCompletion.create(
            engine=MODEL,
            messages = messages,
            temperature=0.25,
            max_tokens=8000,
            stop=None)
    result = response.get('choices',[{}])[0].get('message',{'content':''}).get('content','') 
    try:
        return json.loads(result)
    except:
        # extract json out of the result string
        result = result[result.find('{'):result.rfind('}')+1]
        return json.loads(result)

query("Mt 12:1", 5)

[{'verse': 'Matthew 12:1',
  'question': 'What did the disciples do?',
  'answer': 'They were hungry and began to pluck heads of grain and eat.',
  'examples': {'bad_translation': 'The followers were famished and started to pick fruits and eat.',
   'wrong_answer': 'pick fruits'}},
 {'verse': 'Matthew 12:1',
  'question': 'What day did this happen?',
  'answer': 'The Sabbath',
  'examples': {'bad_translation': 'This occurred on the holy day.',
   'wrong_answer': 'holy day'}},
 {'verse': 'Matthew 12:1',
  'question': 'What did the Pharisees say to Jesus?',
  'answer': 'Look, your disciples are doing what is not lawful to do on the Sabbath.',
  'examples': {'bad_translation': 'Behold, your followers are doing what is forbidden on the holy day.',
   'wrong_answer': 'forbidden on the holy day'}},
 {'verse': 'Matthew 12:1',
  'question': 'Why did the disciples pluck heads of grain?',
  'answer': 'Because they were hungry',
  'examples': {'bad_translation': 'Due to their hunger, the follower